In [5]:
import torch
import math
import torch.nn.functional as F
from torch.autograd import Variable
import torch.nn as nn
from torch import einsum

import torchtext
from torchtext.data.utils import get_tokenizer
from torch.nn import TransformerEncoder, TransformerEncoderLayer


In [9]:

def attention(Q, K, V):
    ''' Functional implementation for scaled dot product attention formula'''
    dot_prod = torch.matmul(Q, torch.transpose(K, -2, -1)) #swap last 2 dims, regardless of batch dim
    K_dim = K.size(-1)
    softmax = F.softmax(dot_prod/math.sqrt(K_dim), dim = -1)
    attention = torch.matmul(softmax, V)
    return attention

class SelfAttentionWide(nn.Module):
    def __init__(self, emb_dim, num_heads):
        super().__init__()
        # we want to output the same dim as embedding to enable residual connection
        # init 3 matrices all the same
        self.M_Q, self.M_V, self.M_K = \
            [nn.Linear(emb_dim, emb_dim*num_heads, bias=False) for _ in range(3)]
        
        self.M_merge_heads = nn.Linear(emb_dim*num_heads, emb_dim, bias=False)
        
    def forward(self, x):
        # get Q, K, V
        Q = x@self.M_Q
        K = x@self.M_K
        V = x@self.M_V 
        multi_att = attention(Q, K, V)
        return self.M_merge_heads(multi_att)
        

class EncoderBlock(nn.Module):
    def __init__(self, emb_dim, num_heads):
        super().__init__()
        self.attention_layer = SelfAttentionWide(emb_dim, num_heads)
        self.layernorm1 = nn.LayerNorm(emb_dim)
        self.layernorm2 = nn.LayerNorm(emb_dim)
        # this can be pretty much any mlp we want
        self.mlp = nn.Sequential(nn.Linear(emb_dim, emb_dim), nn.ReLU())
        
    def forward(self, x):
        x = x + self.attention_layer(x)
        x = self.layernorm1(x)
        x = x + self.mlp(x)
        return self.layernorm2(x)
    
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [16]:
class TransformerModel(nn.Module):
    
    def __init__(self, ntoken, emb_dim, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(emb_dim, dropout)
        
        self.transformer_encoder = nn.Sequential(*[EncoderBlock(emb_dim, nhead) for _ in range(nlayers)])
        self.encoder = nn.Embedding(ntoken, emb_dim)
        self.emb_dim = emb_dim
        self.decoder = nn.Linear(emb_dim, ntoken)

    def forward(self, src):
        src = self.encoder(src) * math.sqrt(self.emb_dim)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output
    

In [7]:

# first let's do data wrangling

TEXT = torchtext.data.Field(tokenize=get_tokenizer("basic_english"),
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)
train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cpu'
def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_txt, batch_size)
val_data = batchify(val_txt, eval_batch_size)
test_data = batchify(test_txt, eval_batch_size)

# params and util fn for training
bptt = 35

def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

ntokens = len(TEXT.vocab.stoi) # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
# model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
# let's train for one batch
ntokens = len(TEXT.vocab.stoi)
data, targets = get_batch(train_data, 0)


In [17]:
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)


In [18]:
model(data)

TypeError: matmul(): argument 'other' (position 1) must be Tensor, not Linear